In [41]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
from tqdm.notebook import tqdm

# Set random seed for reproducibility
SEED = 42
np.random.seed(SEED)

In [42]:
DATA_PATH = './datasets/'

train_features_df = pd.read_csv(os.path.join(DATA_PATH, 'train_features.csv'))
test_features_df = pd.read_csv(os.path.join(DATA_PATH, 'test_features.csv'))

In [43]:
train_features_df.head(2)

,ID,month,B2_mean,B2_std,B2_min,B2_max,B2_median,B3_mean,B3_std,B3_min,...,B11_min,B11_max,B11_median,B12_mean,B12_std,B12_min,B12_max,B12_median,Crop,class
0,ID_t4Tmmn_Jan,Jan,2530.2866,10.403985,2501.0,2600.0,2529.0,2249.9440,24.179565,2187.0,...,1271.0,1497.0,1434.0,2978.0210,241.32278,1977.0,3596.0,2988.0,Rubber,3
1,ID_h14T0B_Jan,Jan,2573.0923,8.352156,2555.0,2605.0,2573.0,2267.8613,15.180915,2220.0,...,1342.0,1446.0,1402.0,2664.9092,129.55461,2061.0,3011.0,2675.0,Rubber,3


In [44]:
print(f"Train dataset shape: {train_features_df.shape}")
print(f"Test dataset shape: {test_features_df.shape}")

Train dataset shape: (7433, 59)
Test dataset shape: (2201, 57)


In [45]:
# Extract base IDs (without month)
train_features_df['base_id'] = train_features_df['ID'].str.split('_').str[1]
test_features_df['base_id'] = test_features_df['ID'].str.split('_').str[1]

# Create submission ID format (ID_XXXXX)
train_features_df['submission_id'] = 'ID_' + train_features_df['base_id']
test_features_df['submission_id'] = 'ID_' + test_features_df['base_id']

print(f"Number of unique IDs in test data: {test_features_df['base_id'].nunique()}")

# Encode target labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_features_df['Crop_encoded'] = label_encoder.fit_transform(train_features_df['Crop'])
print("\nEncoded labels:")
for i, label in enumerate(label_encoder.classes_):
    print(f"{label}: {i}")

Number of unique IDs in test data: 282

Encoded labels:
Cocoa: 0
Palm: 1
Rubber: 2


In [46]:
# Convert the month column to numerical values
month_mapping = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

train_features_df['month_num'] = train_features_df['month'].map(month_mapping)
test_features_df['month_num'] = test_features_df['month'].map(month_mapping)

In [47]:
# Enhanced Feature Engineering
def create_enhanced_features(df):
    """Create advanced features from the spectral bands using band statistics"""
    
    # -------- Basic Vegetation Indices --------
    # NDVI (Normalized Difference Vegetation Index)
    df['NDVI'] = (df['B8_mean'] - df['B4_mean']) / (df['B8_mean'] + df['B4_mean'])
    
    # EVI (Enhanced Vegetation Index)
    df['EVI'] = 2.5 * ((df['B8_mean'] - df['B4_mean']) / (df['B8_mean'] + 6 * df['B4_mean'] - 7.5 * df['B2_mean'] + 1))
    
    # SAVI (Soil Adjusted Vegetation Index)
    L = 0.5  # soil brightness correction factor
    df['SAVI'] = ((df['B8_mean'] - df['B4_mean']) / (df['B8_mean'] + df['B4_mean'] + L)) * (1 + L)
    
    # NDWI (Normalized Difference Water Index)
    df['NDWI'] = (df['B3_mean'] - df['B8_mean']) / (df['B3_mean'] + df['B8_mean'])
    
    # NDBI (Normalized Difference Built-up Index)
    df['NDBI'] = (df['B11_mean'] - df['B8_mean']) / (df['B11_mean'] + df['B8_mean'])
    
    # NBR (Normalized Burn Ratio)
    df['NBR'] = (df['B8_mean'] - df['B12_mean']) / (df['B8_mean'] + df['B12_mean'])
    
    # NDRE (Normalized Difference Red Edge)
    df['NDRE'] = (df['B8_mean'] - df['B5_mean']) / (df['B8_mean'] + df['B5_mean'])
    
    # ------------ Additional Advanced Indices ----------------------
    
    # MTCI (MERIS Terrestrial Chlorophyll Index) - sensitive to chlorophyll content
    df['MTCI'] = (df['B8_mean'] - df['B5_mean']) / (df['B5_mean'] - df['B4_mean'])
    
    # MCARI (Modified Chlorophyll Absorption Ratio Index)
    df['MCARI'] = ((df['B5_mean'] - df['B4_mean']) - 0.2 * (df['B5_mean'] - df['B3_mean'])) * (df['B5_mean'] / df['B4_mean'])
    
    # GNDVI (Green Normalized Difference Vegetation Index)
    df['GNDVI'] = (df['B8_mean'] - df['B3_mean']) / (df['B8_mean'] + df['B3_mean'])
    
    # PSRI (Plant Senescence Reflectance Index) - good for identifying crop maturity
    df['PSRI'] = (df['B4_mean'] - df['B3_mean']) / df['B8_mean']
    
    # SIPI (Structure Insensitive Pigment Index)
    df['SIPI'] = (df['B8_mean'] - df['B2_mean']) / (df['B8_mean'] - df['B4_mean'])
    
    # ARVI (Atmospherically Resistant Vegetation Index)
    df['ARVI'] = (df['B8_mean'] - (2 * df['B4_mean'] - df['B2_mean'])) / (df['B8_mean'] + (2 * df['B4_mean'] - df['B2_mean']))
    
    # MSI (Moisture Stress Index)
    df['MSI'] = df['B11_mean'] / df['B8_mean']
    
    # NDMI (Normalized Difference Moisture Index)
    df['NDMI'] = (df['B8_mean'] - df['B11_mean']) / (df['B8_mean'] + df['B11_mean'])
    
    # CIrededge (Chlorophyll Index Red-Edge)
    df['CIrededge'] = (df['B8_mean'] / df['B5_mean']) - 1
    
    # CIgreen (Chlorophyll Index Green)
    df['CIgreen'] = (df['B8_mean'] / df['B3_mean']) - 1
    
    # Band ratios and combinations
    df['B8_B4'] = df['B8_mean'] / df['B4_mean']
    df['B11_B8'] = df['B11_mean'] / df['B8_mean']
    df['B2_B3'] = df['B2_mean'] / df['B3_mean']
    df['B4_B3'] = df['B4_mean'] / df['B3_mean']
    df['B5_B4'] = df['B5_mean'] / df['B4_mean']
    df['B6_B5'] = df['B6_mean'] / df['B5_mean']
    df['B7_B6'] = df['B7_mean'] / df['B6_mean']
    df['B8A_B8'] = df['B8A_mean'] / df['B8_mean']
    df['B11_B12'] = df['B11_mean'] / df['B12_mean']
    
    # ------- Texture-related features (using std values) --------
    # Variance ratios (using std as a proxy for variance)
    df['var_vis_ratio'] = (df['B2_std'] + df['B3_std'] + df['B4_std']) / 3
    df['var_nir_ratio'] = (df['B8_std'] + df['B8A_std']) / 2
    df['var_swir_ratio'] = (df['B11_std'] + df['B12_std']) / 2
    df['var_rededge_ratio'] = (df['B5_std'] + df['B6_std'] + df['B7_std']) / 3
    
    # Statistical features using existing statistics
    df['mean_vis'] = (df['B2_mean'] + df['B3_mean'] + df['B4_mean']) / 3
    df['mean_nir'] = (df['B8_mean'] + df['B8A_mean']) / 2
    df['mean_swir'] = (df['B11_mean'] + df['B12_mean']) / 2
    df['mean_rededge'] = (df['B5_mean'] + df['B6_mean'] + df['B7_mean']) / 3
    
    df['min_vis'] = np.minimum.reduce([df['B2_min'], df['B3_min'], df['B4_min']])
    df['min_nir'] = np.minimum.reduce([df['B8_min'], df['B8A_min']])
    df['min_swir'] = np.minimum.reduce([df['B11_min'], df['B12_min']])
    df['min_rededge'] = np.minimum.reduce([df['B5_min'], df['B6_min'], df['B7_min']])
    
    df['max_vis'] = np.maximum.reduce([df['B2_max'], df['B3_max'], df['B4_max']])
    df['max_nir'] = np.maximum.reduce([df['B8_max'], df['B8A_max']])
    df['max_swir'] = np.maximum.reduce([df['B11_max'], df['B12_max']])
    df['max_rededge'] = np.maximum.reduce([df['B5_max'], df['B6_max'], df['B7_max']])
    
    df['range_vis'] = df['max_vis'] - df['min_vis']
    df['range_nir'] = df['max_nir'] - df['min_nir']
    df['range_swir'] = df['max_swir'] - df['min_swir']
    df['range_rededge'] = df['max_rededge'] - df['min_rededge']
    
    # Median-based features
    df['median_vis'] = (df['B2_median'] + df['B3_median'] + df['B4_median']) / 3
    df['median_nir'] = (df['B8_median'] + df['B8A_median']) / 2
    df['median_swir'] = (df['B11_median'] + df['B12_median']) / 2
    df['median_rededge'] = (df['B5_median'] + df['B6_median'] + df['B7_median']) / 3
    
    # Median to mean ratios (can indicate skewness)
    df['median_mean_ratio_vis'] = df['median_vis'] / df['mean_vis']
    df['median_mean_ratio_nir'] = df['median_nir'] / df['mean_nir']
    df['median_mean_ratio_swir'] = df['median_swir'] / df['mean_swir']
    df['median_mean_ratio_rededge'] = df['median_rededge'] / df['mean_rededge']
    
    # Seasonal features - sine and cosine transformations
    # Check if month_num exists, otherwise create from month
    if 'month_num' not in df.columns:
        # Assuming month is already a numeric value from 1-12
        df['month_num'] = df['month']
        
    df['month_sin'] = np.sin(2 * np.pi * df['month_num'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month_num'] / 12)
    
    # Season categorical (1=Winter, 2=Spring, 3=Summer, 4=Fall)
    df['season'] = ((df['month_num'] % 12 + 3) // 3) % 4 + 1
    
    # Growth phase approximation (based on month in tropical regions)
    # This is a simplified approximation and we could be refine it later with actual crop calendars
    # For West Africa: roughly planting in April-May, growing in June-Sept, harvesting in Oct-Dec
    conditions = [
        (df['month_num'].isin([4, 5])),  # Planting
        (df['month_num'].isin([6, 7, 8, 9])),  # Growing
        (df['month_num'].isin([10, 11, 12])),  # Harvesting
        (df['month_num'].isin([1, 2, 3]))  # Post-harvest/fallow
    ]
    choices = [1, 2, 3, 4]
    df['growth_phase'] = np.select(conditions, choices, default=0)
    
    # Additional features using std values
    df['std_to_mean_B4'] = df['B4_std'] / df['B4_mean']  # Red band variability
    df['std_to_mean_B8'] = df['B8_std'] / df['B8_mean']  # NIR band variability
    df['std_to_mean_B11'] = df['B11_std'] / df['B11_mean']  # SWIR band variability
    
    # Ratio of max to min (dynamic range)
    df['dynamic_range_B4'] = df['B4_max'] / df['B4_min']
    df['dynamic_range_B8'] = df['B8_max'] / df['B8_min']
    df['dynamic_range_B11'] = df['B11_max'] / df['B11_min']
    
    return df

train_df = create_enhanced_features(train_features_df)
test_df = create_enhanced_features(test_features_df)

In [48]:
def add_missing_features(df):
    """Add features that are missing from the current implementation"""
    
    # 1. Additional Vegetation Indices
    
    # MSAVI (Modified Soil Adjusted Vegetation Index)
    # MSAVI = (2 * NIR + 1 - sqrt((2 * NIR + 1)^2 - 8 * (NIR - RED))) / 2
    df['MSAVI'] = (2 * df['B8_mean'] + 1 - np.sqrt((2 * df['B8_mean'] + 1)**2 - 8 * (df['B8_mean'] - df['B4_mean']))) / 2
    
    # EVI2 (Enhanced Vegetation Index 2) - simpler version of EVI
    df['EVI2'] = 2.5 * ((df['B8_mean'] - df['B4_mean']) / (df['B8_mean'] + 2.4 * df['B4_mean'] + 1))
    
    # IRECI (Inverted Red-Edge Chlorophyll Index)
    df['IRECI'] = (df['B7_mean'] - df['B4_mean']) / (df['B5_mean'] / df['B6_mean'])
    
    # S2REP (Sentinel-2 Red-Edge Position)
    df['S2REP'] = 705 + 35 * ((df['B7_mean'] + df['B4_mean']) / 2 - df['B5_mean']) / (df['B6_mean'] - df['B5_mean'])
    
    # 2. Texture and Variance Metrics
    
    # Coefficient of variation for different spectral regions
    df['cv_vis'] = df[['B2_mean', 'B3_mean', 'B4_mean']].std(axis=1) / df[['B2_mean', 'B3_mean', 'B4_mean']].mean(axis=1)
    df['cv_nir'] = df[['B8_mean', 'B8A_mean']].std(axis=1) / df[['B8_mean', 'B8A_mean']].mean(axis=1)
    df['cv_swir'] = df[['B11_mean', 'B12_mean']].std(axis=1) / df[['B11_mean', 'B12_mean']].mean(axis=1)
    df['cv_rededge'] = df[['B5_mean', 'B6_mean', 'B7_mean']].std(axis=1) / df[['B5_mean', 'B6_mean', 'B7_mean']].mean(axis=1)
    
    # 3. Transformations of existing features
    
    # Log transformations of key bands
    for band in ['B4_mean', 'B8_mean', 'B11_mean']:
        df[f'log_{band}'] = np.log1p(df[band])
    
    # 4. Interaction terms
    
    # Interaction between NDVI and season
    df['NDVI_season'] = df['NDVI'] * df['season']
    
    # Interaction between NDVI and month
    if 'month_num' in df.columns:
        df['NDVI_month'] = df['NDVI'] * df['month_num']
    
    # 5. Additional band ratios and combinations
    
    # Red-edge position indicators
    df['rededge_ratio1'] = df['B5_mean'] / df['B6_mean']
    df['rededge_ratio2'] = df['B6_mean'] / df['B7_mean']
    df['rededge_ratio3'] = df['B5_mean'] / df['B7_mean']
    
    # Ratio between different spectral regions
    df['vis_nir_ratio'] = df['mean_vis'] / df['mean_nir']
    df['nir_swir_ratio'] = df['mean_nir'] / df['mean_swir']
    df['vis_swir_ratio'] = df['mean_vis'] / df['mean_swir']
    
    # 6. LAI approximation
    df['LAI_approx'] = 3.618 * df['EVI'] - 0.118
    
    return df

train_df = add_missing_features(train_df)
test_df = add_missing_features(test_df)

In [49]:
# def create_temporal_features(df):
#     """Create features that capture changes across months for the same location"""
    
#     # Extract base_id from ID if not already present
#     if 'base_id' not in df.columns:
#         df['base_id'] = df['ID'].str.split('_').str[1]
    
#     # Group by base_id
#     grouped = df.groupby('base_id')
    
#     # Create a dictionary to store the new features
#     new_features = {}
    
#     # Initialize the dictionary with empty lists for each new feature
#     for feature in ['NDVI_range', 'NDVI_std', 'NDVI_trend', 'EVI_range', 'EVI_std', 
#                    'B8_range', 'B8_std', 'B4_range', 'B4_std', 'month_count']:
#         new_features[feature] = []
    
#     # Process each base_id group
#     for base_id, group in grouped:
#         # Sort by month_num to ensure chronological order
#         if 'month_num' in group.columns:
#             group = group.sort_values('month_num')
        
#         # Count number of months available
#         month_count = len(group)
        
#         # Calculate range and standard deviation for key indices
#         ndvi_range = group['NDVI'].max() - group['NDVI'].min() if 'NDVI' in group.columns else 0
#         ndvi_std = group['NDVI'].std() if 'NDVI' in group.columns else 0
        
#         evi_range = group['EVI'].max() - group['EVI'].min() if 'EVI' in group.columns else 0
#         evi_std = group['EVI'].std() if 'EVI' in group.columns else 0
        
#         b8_range = group['B8_mean'].max() - group['B8_mean'].min()
#         b8_std = group['B8_mean'].std()
        
#         b4_range = group['B4_mean'].max() - group['B4_mean'].min()
#         b4_std = group['B4_mean'].std()
        
#         # Calculate trend (slope) of NDVI over time
#         ndvi_trend = 0
#         if month_count > 1 and 'NDVI' in group.columns and 'month_num' in group.columns:
#             # Simple linear regression slope calculation
#             x = group['month_num'].values
#             y = group['NDVI'].values
#             slope = np.polyfit(x, y, 1)[0]
#             ndvi_trend = slope
        
#         # Extend the lists with the calculated values (repeated for each row in the group)
#         for _ in range(len(group)):
#             new_features['NDVI_range'].append(ndvi_range)
#             new_features['NDVI_std'].append(ndvi_std)
#             new_features['NDVI_trend'].append(ndvi_trend)
#             new_features['EVI_range'].append(evi_range)
#             new_features['EVI_std'].append(evi_std)
#             new_features['B8_range'].append(b8_range)
#             new_features['B8_std'].append(b8_std)
#             new_features['B4_range'].append(b4_range)
#             new_features['B4_std'].append(b4_std)
#             new_features['month_count'].append(month_count)
    
#     # Add the new features to the dataframe
#     for feature, values in new_features.items():
#         df[feature] = values
    
#     # Calculate additional temporal features
    
#     # Deviation from base_id average for key features
#     for feature in ['NDVI', 'EVI', 'B4_mean', 'B8_mean', 'B11_mean']:
#         if feature in df.columns:
#             # Calculate mean for each base_id
#             feature_mean = df.groupby('base_id')[feature].transform('mean')
#             # Calculate deviation from mean
#             df[f'{feature}_dev_from_mean'] = df[feature] - feature_mean
    
#     # Relative position in the year's cycle for each observation
#     if 'month_num' in df.columns:
#         df['relative_month_position'] = (df['month_num'] - df.groupby('base_id')['month_num'].transform('min')) / \
#                                       (df.groupby('base_id')['month_num'].transform('max') - 
#                                        df.groupby('base_id')['month_num'].transform('min') + 1)
    
#     # Calculate percentile rank of each observation within its base_id group
#     for feature in ['NDVI', 'EVI', 'B4', 'B8']:
#         if feature in df.columns:
#             df[f'{feature}_percentile'] = df.groupby('base_id')[feature].transform(
#                 lambda x: (x.rank(method='min') - 1) / (len(x) - 1) if len(x) > 1 else 0.5
#             )
    
#     return df
    
# train_df = create_temporal_features(train_df)
# test_df = create_temporal_features(test_df)

In [50]:
def add_crop_specific_features(df):
    """Add features specifically designed for crop type discrimination"""
    
    # Crop-specific indices
    
    # TCARI (Transformed Chlorophyll Absorption Ratio Index)
    # Good for chlorophyll content estimation
    df['TCARI'] = 3 * ((df['B5_mean'] - df['B4_mean']) - 0.2 * (df['B5_mean'] - df['B3_mean']) * (df['B5_mean'] / df['B4_mean']))
    
    # MCARI2 (Modified Chlorophyll Absorption Ratio Index Improved)
    # Better for LAI estimation
    df['MCARI2'] = 1.5 * (2.5 * (df['B8_mean'] - df['B4_mean']) - 1.3 * (df['B8_mean'] - df['B3_mean'])) / np.sqrt((2 * df['B8_mean'] + 1)**2 - (6 * df['B8_mean'] - 5 * np.sqrt(df['B4_mean'])) - 0.5)
    
    # WDRVI (Wide Dynamic Range Vegetation Index)
    # Better for high biomass conditions
    a = 0.1  # weighting coefficient
    df['WDRVI'] = (a * df['B8_mean'] - df['B4_mean']) / (a * df['B8_mean'] + df['B4_mean'])
    
    # MTVI2 (Modified Triangular Vegetation Index 2)
    # Good for LAI estimation
    df['MTVI2'] = 1.5 * (1.2 * (df['B8_mean'] - df['B3_mean']) - 2.5 * (df['B4_mean'] - df['B3_mean'])) / np.sqrt((2 * df['B8_mean'] + 1)**2 - (6 * df['B8_mean'] - 5 * np.sqrt(df['B4_mean'])) - 0.5)
    
    # Canopy Chlorophyll Content Index
    df['CCCI'] = ((df['B8_mean'] - df['B5_mean']) / (df['B8_mean'] + df['B5_mean'])) / ((df['B8_mean'] - df['B4_mean']) / (df['B8_mean'] + df['B4_mean']))
    
    # Specific band combinations for crop discrimination
    df['B8_B11_B4'] = df['B8_mean'] / (df['B11_mean'] * df['B4_mean'])
    df['B8_B11_B2'] = df['B8_mean'] / (df['B11_mean'] * df['B2_mean'])
    
    # Texture homogeneity approximation
    df['texture_homogeneity'] = 1 / (1 + df['var_vis_ratio'] + df['var_nir_ratio'] + df['var_swir_ratio'])
    
    return df

train_df = add_crop_specific_features(train_df)
test_df = add_crop_specific_features(test_df)

In [51]:
# Handle potential NaN values from division operations
train_df = train_df.replace([np.inf, -np.inf], np.nan)
test_df = test_df.replace([np.inf, -np.inf], np.nan)
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [52]:
# train_df.head()

### Save the processed data

In [53]:
train_df.isna().sum().sum()

np.int64(0)

In [54]:
'ID_058797' in test_df['submission_id'].values

True

In [55]:
# Save the data
train_df.to_csv(os.path.join(DATA_PATH, "train_df.csv"), index=False)
test_df.to_csv(os.path.join(DATA_PATH, "test_df.csv"), index=False)
print("Data saved successfully")

Data saved successfully
